# Revocation - Bob

## Role: Credential Holder

There will be two parts to this notebook:

- Part 1: Alice issues a revocable credential to [Bob](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb). Bob will store this credential and present proof of it back to Alice while it is not revoked, and Alice will verify the presentation.
- Part 2: Alice then revokes the credential issued to Bob in Part 1 and requests another proof from Bob. This second presentation from Bob will fail to verify.

For details on how revocation works on Hyperledger Indy, you can read more [here](https://github.com/hyperledger/indy-hipe/tree/master/text/0011-cred-revocation).

# Part 1 - Issuing & Proving Non-Revoked Credential

## Complete steps 1-6 in [Alice's revocation notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

## 7. Initialise Bob's controller

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 8. Register listeners

The handler should get called every time the controller receives a webhook with the topic `issue_credential` and `present_proof`, printing out the payload. The agent calls to this webhook every time it receives protocol messages for these topics from a credential.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /aries_basic_controller/aries_controller.py:98>>

In [3]:
def cred_handler(payload):
    print("Handle Credentials")
    global connection_id
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    global pres_ex_id
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([cred_listener, proof_listener], defaults=True)

Handle Credentials
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: offer_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 9. Continue to [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

Alice will issue Bob a revocable credential.

## 10. Check your credential records

You should have one recording the credential offer from Alice. This should also have shown up as as a print statement from the handler function.

In [4]:
response = await agent_controller.issuer.get_records()
results = response["results"]
print(len(results))
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

1
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 11. Request credential from Alice

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the `offer_received` state.

In [5]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: request_sent
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: request_sent
Handle Credentials
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: credential_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 12. Store the redential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the `credential_received` state.

In [6]:
# This can be anything you want. You will use it later to fetch this credential from the agent storage.
credential_id = "My Revocable Credential"

In [7]:
response = await agent_controller.issuer.store_credential(cred_ex_id, credential_id)
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: credential_acked
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]
Credential exchange 2b93329e-5197-4257-865e-d236ebc753ad, role: holder, state: credential_acked


## 13. Check revocation status of the credential

It should not be revoked.

In [8]:
response = await agent_controller.credentials.is_revoked(credential_id)
response

{'revoked': False}

Handle present proof
Role prover, Exchange 949ff847-10ad-45af-9bbd-0d41fc5f78bd in state request_received


## 14. Continue in [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

Send a presentation request for a non revoked credential proof.


## 15. Fetch redentials for presentation

Note `pres_ex_id` is set in the proofs handler.

TODO: make these steps clearer.

In [9]:
pres_ex_id
response = await agent_controller.proofs.get_record_by_id(pres_ex_id)
presentation_request = response['presentation_request']
presentation_request

{'name': 'Proof of Personal Information',
 'version': '1.0',
 'requested_attributes': {'0_skill_uuid': {'name': 'skill',
   'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
   'non_revoked': {'to': 1609949580}}},
 'requested_predicates': {},
 'non_revoked': {'to': 1609949581},
 'nonce': '320470152930602887185694'}

In [10]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")
    
if presentation_request != None:
    # include self-attested attributes (not included in credentials)
    credentials_by_reft = {}
    revealed = {}
    self_attested = {}
    predicates = {}

    # select credentials to provide for the proof
    credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
    print(credentials)

    if credentials:
        for row in sorted(
            credentials,
            key=lambda c: dict(c["cred_info"]["attrs"]),
            reverse=True,
        ):
            for referent in row["presentation_referents"]:
                if referent not in credentials_by_reft:
                    credentials_by_reft[referent] = row

    for referent in presentation_request["requested_attributes"]:
        if referent in credentials_by_reft:
            revealed[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ],
                "revealed": True,
            }

        else:
            self_attested[referent] = "South Africa"

    for referent in presentation_request["requested_predicates"]:
        if referent in credentials_by_reft:
            predicates[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ]
            }

    print("\nGenerate the proof")
    proof = {
        "requested_predicates": predicates,
        "requested_attributes": revealed,
        "self_attested_attributes": self_attested,
    }
    print(proof)
    print("\nXXX")
    print("\npredicates:\n{}".format(predicates))
    print("\nrevealed:\n{}".format(revealed))
    print("\nself_attested:\n{}".format(self_attested))
    
else: 
    print("No presenation record identifier")

[{'cred_info': {'referent': 'My Revocable Credential', 'attrs': {'name': 'Bob', 'skill': 'researcher', 'age': '21'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default:CL_ACCUM:4df26d09-f517-4033-a52f-ca545b9082b3', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1609949580}, 'presentation_referents': ['0_skill_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX

predicates:
{}

revealed:
{'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}

self_attested:
{}


## 16. Send presentation

Bob sends the proof presentation to Alice for verification.

In [11]:
response = await agent_controller.proofs.send_presentation(pres_ex_id, proof)
print(response)

{'presentation_exchange_id': '949ff847-10ad-45af-9bbd-0d41fc5f78bd', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'skill': '24214165878429772374367858032287880675266648921286067519661391650938186682695'}, 'a_prime': '54983037763240015109173480340440884486183126767870275833537707020067559599249365570029595363987274353273341618618272187737385627458909880023440083865568093885337082639401577411997053177445788484601610955542587301251048297570365585153302915148072164865771039331663067917860409305021787753346091792958965893361695899586474121176659153337629782490099179292784354302715333893621954400296724760615557450988515080933827458101229013693435163501896535153968816079537024837255146812669105967485682854386126279373269585547115641271724211401401410035253556300061751314394401534939196433175809773705102317410643077170215493216949', 'e': '73197406319770472061853449902401888496312162477492713604044524574703806668335407371007170030643827385184752890101

Handle present proof
Role prover, Exchange 949ff847-10ad-45af-9bbd-0d41fc5f78bd in state presentation_sent
Handle present proof
Role prover, Exchange 949ff847-10ad-45af-9bbd-0d41fc5f78bd in state presentation_acked


## 17. Continue in [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

Alice will verify the non-revoked credential to be OK.

# Part 2 - Revoking the credential

## 19. Check credential

Bob will check his credential now to find that it is revoked.

In [12]:
response = await agent_controller.credentials.is_revoked(credential_id)
response

{'revoked': True}

Handle present proof
Role prover, Exchange d8d98b64-690d-480d-ad1e-8d5bc6470453 in state request_received


## 20. Continue in [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

Alice will request for a credential proof once again.

## 21. Fetch credentials for presentation proof

Note `pres_ex_id` is set in the proofs handler.

In [13]:
# fetch credentials for presenting proof
pres_ex_id
response = await agent_controller.proofs.get_record_by_id(pres_ex_id)
presentation_request = response['presentation_request']
presentation_request

{'name': 'Proof of Personal Information',
 'version': '1.0',
 'requested_attributes': {'0_skill_uuid': {'name': 'skill',
   'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
   'non_revoked': {'to': 1609949705}}},
 'requested_predicates': {},
 'non_revoked': {'to': 1609949706},
 'nonce': '714708092518710934372484'}

In [14]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")
    
if presentation_request != None:
    # include self-attested attributes (not included in credentials)
    credentials_by_reft = {}
    revealed = {}
    self_attested = {}
    predicates = {}

    # select credentials to provide for the proof
    credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
    print(credentials)

    if credentials:
        for row in sorted(
            credentials,
            key=lambda c: dict(c["cred_info"]["attrs"]),
            reverse=True,
        ):
            for referent in row["presentation_referents"]:
                if referent not in credentials_by_reft:
                    credentials_by_reft[referent] = row

    for referent in presentation_request["requested_attributes"]:
        if referent in credentials_by_reft:
            revealed[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ],
                "revealed": True,
            }

        else:
            self_attested[referent] = "South Africa"

    for referent in presentation_request["requested_predicates"]:
        if referent in credentials_by_reft:
            predicates[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ]
            }

    print("\nGenerate the proof")
    proof = {
        "requested_predicates": predicates,
        "requested_attributes": revealed,
        "self_attested_attributes": self_attested,
    }
    print(proof)
    print("\nXXX")
    print("\npredicates:\n{}".format(predicates))
    print("\nrevealed:\n{}".format(revealed))
    print("\nself_attested:\n{}".format(self_attested))
    
else: 
    print("No presenation record identifier")

[{'cred_info': {'referent': 'My Revocable Credential', 'attrs': {'name': 'Bob', 'skill': 'researcher', 'age': '21'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default:CL_ACCUM:4df26d09-f517-4033-a52f-ca545b9082b3', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1609949705}, 'presentation_referents': ['0_skill_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX

predicates:
{}

revealed:
{'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}

self_attested:
{}


## 22. Send presentation

Bob sends the proof presentation to Alice for verification.

In [15]:
response = await agent_controller.proofs.send_presentation(pres_ex_id, proof)
print(response)

{'presentation_exchange_id': 'd8d98b64-690d-480d-ad1e-8d5bc6470453', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'skill': '24214165878429772374367858032287880675266648921286067519661391650938186682695'}, 'a_prime': '75832767533944430921040650558001002566293763366725659023861160961464313787612907826354361589543604831848702439617797974347354598118482576327042650181393101025629552520760211933523781694847530595035216377080758008777044854372152718615572516472187175883477040130911167731948080394027232014494581911887353545213285448856086776923724705656170762349423754937886349069555698503872339493536942744687164145352805316459258250097038956045990529900118367803303464509120110388827779724304557816144346842789519895415571462968875285968081140783812282296797865863124288954867313619081346765221166855508234020246259261155064658134649', 'e': '66863441126845964068603707085386192147709547452718771813053867868059488518178037062632322439922320426573278275142

Handle present proof
Role prover, Exchange d8d98b64-690d-480d-ad1e-8d5bc6470453 in state presentation_sent
Handle present proof
Role prover, Exchange d8d98b64-690d-480d-ad1e-8d5bc6470453 in state presentation_acked


## 23. Continue in [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

Alice will verify the now revoked credential to be NOT OK.

## End of Tutorial

Be sure to terminate the controller.

In [ ]:
response = await agent_controller.terminate()
print(response)